In [11]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
car=pd.read_csv('audi.csv')
car.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


# checking data and data manipulation

In [78]:
print(car.shape)
columns=['year','price','mileage','tax','mpg','engineSize']
for i in columns:
    print(car[i].dtype,end=' ')
    print(np.isnan(car[i]).unique())
car['model'].unique()

(10668, 9)
int64 [False]
int64 [False]
int64 [False]
int64 [False]
float64 [False]
float64 [False]


array([' A1', ' A6', ' A4', ' A3', ' Q3', ' Q5', ' A5', ' S4', ' Q2',
       ' A7', ' TT', ' Q7', ' RS6', ' RS3', ' A8', ' Q8', ' RS4', ' RS5',
       ' R8', ' SQ5', ' S8', ' SQ7', ' S3', ' S5', ' A2', ' RS7'],
      dtype=object)

In [44]:
car.describe()

,year,price,mileage,tax,mpg,engineSize
count,10668.000000,10668.000000,10668.000000,10668.000000,10668.000000,10668.000000
mean,2017.100675,22896.685039,24827.244001,126.011436,50.770022,1.930709
std,2.167494,11714.841888,23505.257205,67.170294,12.949782,0.602957
min,1997.000000,1490.000000,1.000000,0.000000,18.900000,0.000000
25%,2016.000000,15130.750000,5968.750000,125.000000,40.900000,1.500000
50%,2017.000000,20200.000000,19000.000000,145.000000,49.600000,2.000000
75%,2019.000000,27990.000000,36464.500000,145.000000,58.900000,2.000000
max,2020.000000,145000.000000,323000.000000,580.000000,188.300000,6.300000


In [57]:
# To check whether there is any outlier present in the data that we should remove:
car[car['price']<1e4]
# based on checking there isn't

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
30,A1,2015,9500,Automatic,54754,Diesel,0,76.3,1.6
60,A1,2013,7000,Manual,76269,Diesel,0,74.3,1.6
73,A3,2016,8300,Manual,88215,Diesel,20,68.9,2.0
117,A1,2016,9800,Manual,36462,Diesel,0,76.3,1.6
147,A4,2013,9400,Manual,39998,Diesel,30,65.7,2.0
...,...,...,...,...,...,...,...,...,...
10650,A4,2011,6995,Manual,88000,Diesel,30,61.4,2.0
10651,A3,2011,9695,Manual,32300,Petrol,235,39.2,2.0
10652,A1,2014,9995,Manual,54000,Petrol,30,55.4,1.2
10660,A4,2011,9995,Automatic,78000,Diesel,305,39.8,3.0


In [55]:
y=car['price']
X=car.drop(columns=['price'])
X.head()

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,Manual,1998,Petrol,145,49.6,1.0


In [62]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [93]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge 
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [85]:
scaler=MinMaxScaler()
ohe=OneHotEncoder()
ohe.fit(X[['model','year','transmission','fuelType']])
lr=LinearRegression()
# Now i'll do sort of column transformation 
# here i'm passing all categories to onehotencoder so that it knows already,helpful when suppose X_test have some categories that doesn't have in X_test

column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['model','year','transmission','fuelType']),(scaler,['mileage']),remainder='passthrough')
# Now i'll apply transformers 
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array([' A1', ' A2', ' A3', ' A4', ' A5', ' A6', ' A7', ' A8', ' Q2',
       ' Q3', ' Q5', ' Q7', ' Q8', ' R8', ' RS3', ' RS4', ' RS5', ' RS6',
       ' RS7', ' S3', ' S4', ' S5', ' S8', ' SQ5', ' SQ7', ' TT'],
      dtype=object),
                                                                            array([1997, 1998, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=int64),
                                                                            array(['Automatic', 'Manual', 'Semi-Auto'], dtype=object),
                                                                            array(['Diesel', 'Hybrid', 'Petrol'], dtype=object)]),
                        

In [86]:
y_pred=pipe.predict(X_test)
y_pred.shape

(2667,)

In [87]:
r2_score(y_test,y_pred)

0.9023824496544428

In [ ]:
## comparison between ridge_reg and linear_reg results:

In [98]:
list1=[0.002,0.002,0.2,2.0,4.0,10.0,15.0,25.0,30.0,40.0,100.0]
score=[]
for i in list1:
    ridge=Ridge(alpha=i)
    pipe2=make_pipeline(column_trans,ridge)
    pipe2.fit(X_train,y_train)
    y_pred_ridge=pipe2.predict(X_test)
    score.append(r2_score(y_test,y_pred_ridge))
print(score)

[0.873623949804532, 0.873623949804532, 0.8735520794156187, 0.8727657298211566, 0.8637031619690548, 0.8603754493591143, 0.8262673048509925, 0.8241388869553272, 0.8229435542164057, 0.8065263264759117, 0.7868480775186124]
